# Physical units

All variables in Scipp have a physical unit.
Variables are used for coordinates, data, and attributes, therefore, all of these have a unit.

## Basic Operations

Units are encoded by the [scipp.Unit](../generated/classes/scipp.Unit.rst) class.
Instances of this class can be constructed from strings:

In [ ]:
import scipp as sc

length = sc.Unit('m')
length

[scipp.Unit](../generated/classes/scipp.Unit.rst) defines mathematical operators for combining units:

In [ ]:
area = length * length
area

In [ ]:
volume = length * length * length
volume

In [ ]:
also_volume = length ** 3
also_volume

In [ ]:
sc.Unit('dimensionless') / length

In [ ]:
speed = length / sc.Unit('s')
speed

Invalid operations raise exceptions:

In [ ]:
speed + length

It is also possible to construct composite units directly from strings:

In [ ]:
sc.Unit('km')

In [ ]:
sc.Unit('m/s')

In [ ]:
sc.Unit('counts')

In [ ]:
sc.Unit('kg*m^2/s^2')

For convenience, the [scipp.units](../generated/modules/scipp.units.rst) module provides some frequently used units.
See [scipp.units](../generated/modules/scipp.units.rst) for a list of those units.

In [ ]:
sc.units.kg

In [ ]:
sc.units.m / sc.units.s

In [ ]:
sc.units.dimensionless

Use `repr` to see a unit in terms of SI (plus extensions) base units:

In [ ]:
repr(sc.Unit('V/L'))

This is especially helpful when it is unclear what a particular unit represents.

## Constructing Variables with Units

[Variables](../generated/classes/scipp.Variable.rst#scipp.Variable) with units can be constructed using the `units` argument in the constructor or in [creation functions](./creation-functions.rst).
When not specified explicitly, the unit of a variable defaults to `dimensionless` (a.k.a. `one`).
That is, the variable is considered dimensionless in terms of units (not to be confused with array dimensions).

In [ ]:
# same as sc.Variable(dims=['x'], values=[1, 2])
# and     sc.Variable(dims=['x'], values=[1, 2], unit='dimensionless')
sc.Variable(dims=['x'], values=[1, 2], unit='one')

In [ ]:
sc.Variable(dims=['x'], values=[1, 2], unit='m')

In [ ]:
sc.Variable(dims=['x'], values=[1, 2], unit=sc.units.m)

In [ ]:
sc.arange('x', 0, 3, unit=sc.units.s)

Scalars can also be constructed using multiplication or division of a number and a unit (in addition to [scipp.scalar](../generated/functions/scipp.scalar.rst#scipp.scalar)):

In [ ]:
1.2 * sc.Unit('kg/m^3')

In [ ]:
3.4 / sc.units.K

## Supported Units

Scipp supports a great number of units through LLNL's [Units](https://units.readthedocs.io/en/latest/index.html) library.
See in particular [Defined Units](https://units.readthedocs.io/en/latest/user-guide/defined_units.html).

<div class="alert alert-info">
     <b>INFO</b>

The LLNL/Units library is considered an implementation detail of Scipp.
Using SI units is safe but other unit systems should be used with discretion.
This applies especially to non-standard units like LLNL/Unit's custom (counting) units.
</div>

### Base Units
All SI base units are supported with the following names:

| Name  | Unit     |
|-------|----------|
| 'm'   | meter    |
| 's'   | second   |
| 'kg'  | kilogram |
| 'K'   | kelvin   |
| 'A'   | ampere   |
| 'mol' | mole     |
| 'cd'  | candela  |

In addition, the following base units are supported for cases not covered by SI.

| name    | Unit                   |
|---------|------------------------|
| 'rad'   | radian                 |
| 'count' | single object counting |

### Derived units
Many derived units can also be specified as arguments to `sc.Unit`.
Some examples are

| Name             | Unit          |
|------------------|---------------|
| 'Hz'             | hertz         |
| 'J'              | joule         |
| 'V'              | volt          |
| 'W'              | watt          |
| 'angstrom' / 'Å' | ångström     |
| 'eV'             | electron volt |
| 'L'              | liter         |
| 'min'            | minute        |
| 'D' / 'day'      | day           |

Units can be modified with SI prefixes, for instance

In [ ]:
print(sc.Unit('mm'), sc.Unit('microsecond'),
      sc.Unit('micro s'), sc.Unit('us'), sc.Unit('MJ'))

You can also specify exponents for units or exponentiate the `Unit` object:

In [ ]:
print(sc.Unit('m^2'), sc.Unit('m**2'), sc.Unit('m')**2)

## Conversion Between Units of Different Scales

Data can be converted between compatible units using [sc.to_unit](../generated/functions/scipp.to_unit.rst#scipp.to_unit).
Only conversions between units of the same physical dimensions are possible.

In [ ]:
sc.to_unit(1.0 * sc.units.m, 'mm')

In [ ]:
sc.to_unit(1.0 * sc.Unit('parsec'), 'm')

In [ ]:
sc.to_unit(3.14 * sc.Unit('m/s'), 'km/h')

In [ ]:
sc.to_unit(1.0 * sc.Unit('s'), 'm')

# Unit Aliases

It is possible to define custom aliases for units.
This can be used to

- guide string formatting to prefer a certain unit, e.g. angstrom over nm
- define domain specific units that can be expressed in terms of other units to
  - guide string formatting
  - construct units from strings with custom names
  
## Prioritizing Units in String Formatting

When dealing with crystals or molecules, it is often convenient to use angstrom as a unit.
But by default, string formatting tends to prefer different bases in composite units.
For example

In [ ]:
sc.Unit('us/angstrom**2')

This result is not very useful.
We can make the formatter prefer angstrom by defining it as an alias of itself (or alternatively of `'10^-10 m'`):

In [ ]:
sc.units.aliases['angstrom'] = 'angstrom'
sc.Unit('us/angstrom**2')

Aliases are global and stay in effect until they are removed.
This can be done using `sc.units.aliases.clear()` to remove all aliases or, to remove only one, using

In [ ]:
del sc.units.aliases['angstrom']
sc.Unit('us/angstrom**2')

Alternatively, a context manager can be used to remove aliases automatically:

In [ ]:
with sc.units.aliases.scoped(angstrom='angstrom'):
    print(sc.Unit('us/angstrom**2'))
print(sc.Unit('us/angstrom**2'))

But note that the context manager uses the global alias table and affects code outside of the context.
Details are explained in [scipp.units.aliases.scoped](../generated/modules/scipp.units.UnitAliases.rst#scipp.units.UnitAliases.scoped).

See [scipp.units.UnitAliases](../generated/modules/scipp.units.UnitAliases.rst) for the full API of `sc.units.aliases`.

## Defining New Units

It is possible to define completely new units as aliases as long as they can be expressed in terms of other units.
For example, define the horse length:

In [ ]:
sc.units.aliases['horse'] = sc.scalar(2.4, unit='m')

Alternatively, we can pass `'2.4 m'` instead of a variable.

The string formatter now prioritizes 'horse' over 'm' when appropriate:

In [ ]:
print(sc.Unit('2.4m'))
print(sc.Unit('2.4m/s'))
print(sc.Unit('m'))

And we can also construct units using 'horse' as an argument:

In [ ]:
print(sc.Unit('horse'))
print(sc.Unit('mhorse**2/g'))

This also works in variables:

In [ ]:
sc.scalar(4, unit='horse')

In [ ]:
sc.scalar(4, unit='horse').to(unit='m')

Using `repr`, we can see that 'horse' is not a fundamentally new unit but simply expressed in terms of 'm':

In [ ]:
repr(sc.Unit('horse'))